AdaBoostClassifier - OK
BaggingClassifier
BernoulliNB
CalibratedClassifierCV
DecisionTreeClassifier - OK
ExtraTreeClassifier
ExtraTreesClassifier
GaussianNB - OK
GaussianProcessClassifier
GradientBoostingClassifier - OK
KNeighborsClassifier - OK
LabelPropagation
LabelSpreading
LinearDiscriminantAnalysis
LinearSVC
LogisticRegression - OK
LogisticRegressionCV
MLPClassifier - OK
MultinomialNB
NearestCentroid
NuSVC
PassiveAggressiveClassifier
Perceptron
QuadraticDiscriminantAnalysis - OK
RadiusNeighborsClassifier
RandomForestClassifier - OK
RidgeClassifier
RidgeClassifierCV
SGDClassifier
SVC - OK

In [71]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

print(__doc__)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split


from time import time
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import RobustScaler, StandardScaler, LabelEncoder

import warnings
warnings.filterwarnings('ignore')


Automatically created module for IPython interactive environment


In [37]:
df = pd.read_csv("top_products.csv")

In [38]:
df['target'] = np.random.randint(2, size=len(df))


In [39]:
df.head()

,Top1,Top2,Top3,Top4,Preco,Quantidade,Promotion,Receita,Clima,target
0,Carbonated Water - Blackberry,Soup - Campbells Tomato Ravioli,Wine - Crozes Hermitage E.,Bread - Sour Batard,17,7,1,234,3,1
1,Puff Pastry - Slab,Soup - Campbells Mushroom,Gatorade - Cool Blue Raspberry,Cookie - Oreo 100x2,78,2,1,318,1,1
2,Wine - Stoneliegh Sauvignon,Momiji Oroshi Chili Sauce,Vodka - Moskovskaya,Coffee - Almond Amaretto,100,4,1,476,2,1
3,Baking Soda,Icecream - Dstk Cml And Fdg,"Artichoke - Bottom, Canned",Bread - Hamburger Buns,40,6,1,438,2,1
4,Wine - Soave Folonari,Couscous,Bread - Pumpernickel,Wine - Casillero Del Diablo,61,2,1,492,4,1


In [40]:
y = df['target'].values
X = df.drop(['target'], axis=1)

In [41]:
class MultiColumnLabelEncoder:
    
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)


In [42]:
transform = MultiColumnLabelEncoder()

In [43]:
X = transform.transform(X)

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [46]:
names = ["Bagging", "Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA", "Logistic Regression", "Gradient Boosting", "Neural Net"]


In [63]:

classifiers = [
    BaggingClassifier(),
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis(),
    LogisticRegression(),
    GradientBoostingClassifier(n_estimators=50),
    MLPClassifier(alpha=1)
                            ]


In [76]:
class Classifiers():
    
    def __init__(self):
        pass
    
    def training(self, X_train, y_train, X_test, y_test):
        """
        fitting the traing set in the models
        """
        lista = []
        for name, clf in zip(names, classifiers):
            t_start = time.clock()
            fit = clf.fit(X_train, y_train)
            t_end = time.clock()
            t_diff = t_end - t_start
            score_test = fit.score(X_test, y_test)
            score_train = fit.score(X_train, y_train)
            results = (name, score_test, score_train, t_diff)
            lista.append(results)
        results = sorted(lista, key=lambda x: x[1], reverse=True)
        for a, b in results:
            print ("Modelo", a,"tem score igual a: {:.1%}".format(b))
            
    def trainingtime(self, X_train, y_train, X_test, y_test):
        """
        fitting the traing set in the models
        """
        lista = []
        for name, clf in tqdm_notebook(zip(names, classifiers), desc="Running:"):
            t_start = time()
            fit = clf.fit(X_train, y_train)
            t_end = time()
            t_diff = t_end - t_start
            score_test = fit.score(X_test, y_test)
            score_train = fit.score(X_train, y_train)
            results = (name, score_test, score_train, t_diff)
            lista.append(results)
        results = sorted(lista, key=lambda x: x[1], reverse=True)
        labels = ['Algoritmo', 'Test Score', 'Train Score', 'Training Time']
        df = pd.DataFrame.from_records(results, columns=labels)
        return df

In [77]:
classificadores = Classifiers()

In [78]:
beta = classificadores.trainingtime(X_train, y_train, X_test, y_test)

HBox(children=(IntProgress(value=1, bar_style='info', description='Running Algorithms', max=1), HTML(value='')))

In [75]:
beta

,Algoritmo,Test Score,Train Score,Training Time
0,RBF SVM,0.545,1.00000,0.019366
1,QDA,0.540,0.49625,0.001161
2,Gradient Boosting,0.530,0.81250,0.049756
3,Random Forest,0.515,0.71750,0.015693
4,Naive Bayes,0.510,0.55500,0.001468
5,Linear SVM,0.500,0.54250,2.454969
6,Neural Net,0.495,0.58250,0.142615
7,Logistic Regression,0.490,0.54125,0.005864
8,Decision Tree,0.470,0.65875,0.003385
9,Neural Net,0.465,0.59000,0.147623
